# Predicting with Exponential Moving Averages

I've had some success predicting swings by eyeballing exponential moving averages (EMAs). Now I want to use that idea to backtest against months of PancakeSwap's BNB prediction game.

In [8]:
import os
import sys
import web3
import time
import json
import pandas
import requests

## OHLC Data

Implement a class that obtains OHLC data from whatever source, can save/load as JSON, and can update on demand.

In [48]:
class OHLCData(dict):
    def load(self):
        pass
    def save(self):
        pass
    def retrieve(self):
        pass
    
        


## Statistics: Exponential Moving Averages

Find a way to efficiently go through the OHLC data, collecting the following statistics:
* EMA(high, 60)
* EMA(high, 20)
* EMA(high, 10)
* EMA(low,  10)
* EMA(low,  20)
* EMA(low,  60)


## Prediction Strategies

Implement various prediction strategies to be backtested against the data.

In [2]:
def strategy_biggest(round_info, ohlc):
    if round_info.payout_up > round_info.payout_down:
        return 'up'
    elif round_info.payout_up < round_info.payout_down:
        return 'down'
    return None

def strategy_smallest(round_info, ohlc):
    if round_info.payout_up > round_info.payout_down:
        return 'down'
    elif round_info.payout_up < round_info.payout_down:
        return 'up'
    return None

def strategy_bull(round_info, ohlc):
    return 'up'

def strategy_bear(round_info, ohlc):
    return 'down'

strategies = [strategy_biggest, strategy_smallest, strategy_bull, strategy_bear]

## PancakeSwap Prediction Data

Implement a class that obtains data about past Prediction games on PancakeSwap, can save/load as JSON, and can update on demand.


In [50]:
class PredictionData(dict):
    pks_contract_addr = web3.Web3.toChecksumAddress('0x18b2a687610328590bc8f2e5fedde3b582a49cda')
    bsc_data_url      = 'https://bsc-dataseed.binance.org/'
    bsc_api_url       = 'https://api.bscscan.com/api'
    connection        = None
    pks_contract      = None
    round_data        = None
    
    def save(self):
        pass
    
    def load(self):
        pass
    
    def connect(self):
        connection = web3.Web3(web3.Web3.HTTPProvider(self.bsc_data_url))
        if not connection.isConnected():
            self.connection = None
            return False
        self.connection = connection
        return True
    
    def load_contract(self):
        abi_url           = f'{self.bsc_api_url}?module=contract&action=getabi&address={self.pks_contract_addr}'
        rr                = requests.get(url = abi_url)
        self.abi          = json.loads(rr.json()['result'])
        self.pks_contract = self.connection.eth.contract(address= self.pks_contract_addr, abi= self.abi)
        self.func_inputs  = {func['name']: func[ 'inputs'] for func in self.abi if 'name' in func and  'inputs' in func}
        self.func_outputs = {func['name']: func['outputs'] for func in self.abi if 'name' in func and 'outputs' in func}
        return
    
    def get_round(self, index):
        func = self.pks_contract.functions.rounds(index) 
        rlist = func.call()
        return {spec['name']: value for value, spec in zip(rlist, self.func_outputs['rounds'])}
    
    def update(self):
        if self.round_data is None:
            self.round_data = {}
        for ii in range(epoch):
            if ii not in self.round_data:
                self.round_data[ii] = self.get_round(ii)
        return


In [47]:
pred = PredictionData()
print(pred.pks_contract_addr)
pred.connect()
pred.load_contract()
now = pred.pks_contract.functions.currentEpoch().call()
print(now)
pprint.pprint(pred.get_round(now))
pprint.pprint(pred.get_round(now - 1))
#pred.retrieve_data()
print('-'*80)
#func = pred.pks_contract.functions.rounds(now-1) 
#rlist = func.call()


0x18B2A687610328590Bc8F2e5fEdDe3b582A49cdA
43826
{'bearAmount': 0,
 'bullAmount': 15980300212943869,
 'closeOracleId': 0,
 'closePrice': 0,
 'closeTimestamp': 1644478234,
 'epoch': 43826,
 'lockOracleId': 0,
 'lockPrice': 0,
 'lockTimestamp': 1644477934,
 'oracleCalled': False,
 'rewardAmount': 0,
 'rewardBaseCalAmount': 0,
 'startTimestamp': 1644477634,
 'totalAmount': 15980300212943869}
{'bearAmount': 20816517738374248164,
 'bullAmount': 15577280856444978882,
 'closeOracleId': 0,
 'closePrice': 0,
 'closeTimestamp': 1644477934,
 'epoch': 43825,
 'lockOracleId': 18446744073710488939,
 'lockPrice': 41979000000,
 'lockTimestamp': 1644477625,
 'oracleCalled': False,
 'rewardAmount': 0,
 'rewardBaseCalAmount': 0,
 'startTimestamp': 1644477325,
 'totalAmount': 36393798594819227046}
--------------------------------------------------------------------------------


## Pulling it Together

Implement a class or function that loads/updates the OHLC data calculates the statistics on the OHLCs, loads/updates the prediction data, and then backtests a list of prediction strategies, presenting the results in tabular form.